In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df_train = pd.read_csv('/kaggle/input/google-stock-price/Google_Stock_Price_Train.csv')
df_train.head(n=5)

In [ ]:
plt.plot(df_train.Open)
plt.show()

In [ ]:
def create_dataset(sequence,look_back=60):
    x, y = [],[]
    for i in range(len(sequence) - look_back):
        x_temp = sequence[i:i+look_back]
        y_temp = sequence[i+look_back]
        x.append(x_temp)
        y.append(y_temp)
    return np.array(x),np.array(y)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_train_open = scaler.fit_transform(df_train[['Open']])

In [ ]:
x_train,y_train = create_dataset(df_train_open,60)

In [ ]:
x_train.shape,y_train.shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import *

In [ ]:
model = Sequential()
model.add(LSTM(50, input_shape=x_train.shape[1:],return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mse',optimizer='adam',metrics=['accuracy'])

In [ ]:
checkpoint = ModelCheckpoint('stock_price_model.h5',monitor='val_loss',verbose=1,save_best_only=True)

In [ ]:
hist = model.fit(x_train,y_train,validation_split=0.1,batch_size=64,epochs=50,callbacks=[checkpoint])

In [ ]:
plt.plot(hist.history['loss'],label='Loss')
plt.plot(hist.history['val_loss'],label='Loss')
plt.show()

In [ ]:
y_train_predicted = model.predict(x_train)
y_train_predicted.shape

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(y_train,'r')
plt.plot(y_train_predicted,'b')

In [ ]:
df_test = pd.read_csv('../input/google-stock-price/Google_Stock_Price_Test.csv')

In [ ]:
df_test.head(n=5)

In [ ]:
plt.plot(df_test.Open)
plt.show()

In [ ]:
last_train = df_train.Open[-60:]

In [ ]:
test_open = pd.concat((last_train,df_test.Open),axis=0)
test_open

In [ ]:
test_open_scaled = scaler.fit_transform(np.array(test_open).reshape(-1,1))

In [ ]:
test_open_scaled.shape

In [ ]:
x_test, y_test = create_dataset(test_open_scaled, 60)

In [ ]:
x_test.shape, y_test.shape

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
loaded_model = load_model('./stock_price_model.h5')
predicted_test_open = loaded_model.predict(x_test)

In [ ]:
predicted_test_open = scaler.inverse_transform(predicted_test_open)

In [ ]:
plt.plot(predicted_test_open, 'r')
plt.plot(df_test.Open, 'b')
plt.show()